In [1]:
pip install qiskit -q

Note: you may need to restart the kernel to use updated packages.


# Perturbed Graph States

In the following notebook we construct **perturbed graph states** which are graph states using controlled random unitary matrix gates to entangle qubits connected by an edge in the graph of the graph state. Once we have such states we can use them to study the persistent homology of entanglement. 

## Using Qiskit

In [2]:
import networkx as nx
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.extensions import UnitaryGate
from qiskit.quantum_info.random import random_unitary

def graph_state_circuit(num_qubits):
    """Returns a graph state circuit on num_qubits qubits"""
    # Generate a random graph with num_qubits nodes
    graph = nx.fast_gnp_random_graph(num_qubits, 0.5)

    # Initialize the circuit with the given qubits
    qubits = QuantumRegister(num_qubits)
    circuit = QuantumCircuit(qubits)

    # Apply Hadamard gates to all qubits
    circuit.h(qubits)

    # Apply random controlled unitary gates to each edge
    for edge in graph.edges():
        control_qubit, target_qubit = qubits[edge[0]], qubits[edge[1]]

        # Generate a random unitary matrix
        random_unitary_matrix = random_unitary(2).data

        # Apply the controlled random unitary gate to the circuit
        controlled_unitary_gate = UnitaryGate(random_unitary_matrix).control()
        circuit.append(controlled_unitary_gate, [control_qubit, target_qubit])

    # Define number of qubits in circuit
    # circuit.num_qubits = len(circuit.qubits)

    return circuit


In [3]:
num_qubits = 4
circuit = graph_state_circuit(num_qubits)
circuit.draw()

┌───┐                                 
q0_0: ┤ H ├─────■──────────■────────────────
      ├───┤┌────┴────┐     │                
q0_1: ┤ H ├┤ Unitary ├─────┼──────────■─────
      ├───┤└─────────┘     │          │     
q0_2: ┤ H ├────────────────┼──────────┼─────
      ├───┤           ┌────┴────┐┌────┴────┐
q0_3: ┤ H ├───────────┤ Unitary ├┤ Unitary ├
      └───┘           └─────────┘└─────────┘

## Using Cirq

In [4]:
import cirq
import numpy as np

def graph_state_circuit(num_qubits):
    """Returns a graph state circuit on num_qubits qubits"""
    # Generate a random graph with num_qubits nodes
    graph = nx.fast_gnp_random_graph(num_qubits, 0.5)

    # Initialize the circuit with the given qubits
    qubits = [cirq.LineQubit(i) for i in range(num_qubits)]
    circuit = cirq.Circuit()

    # Apply Hadamard gates to all qubits
    circuit.append(cirq.H.on_each(qubits))

    # Apply random controlled unitary gates to each edge
    for edge in graph.edges():
        control_qubit, target_qubit = qubits[edge[0]], qubits[edge[1]]

        # Generate a random unitary matrix
        random_unitary = cirq.testing.random_unitary(2)

        # Apply the controlled random unitary gate to the circuit
        circuit.append(cirq.ControlledGate(cirq.MatrixGate(random_unitary)).on(control_qubit, target_qubit))

        
    # Define number of qubits in circuit
    circuit.num_qubits = len(circuit.all_qubits())

    
    return circuit

In [5]:
num_qubits = 4
circuit = graph_state_circuit(num_qubits)
circuit

┌────────────────────────────────────────────────────────┐
0: ───H────@───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
           │
1: ───H────┼──────────────────────────@────────────────────────────────@───────────────────────────────────────────────────────────────
           │                          │                                │
           │                          ┌                           ┐    │
2: ───H────┼──────────────────────────│ 0.681-0.143j  0.043+0.717j│────┼───────────────────────────────@───────────────────────────────
           │                          │ 0.158-0.701j -0.649-0.251j│    │                               │
           │                          └                           ┘    │                               │
           │                                                           │                               │
           ┌                         ┐                                 ┌                           ┐   ┌                           ┐
3: ───H────│0.578-0.795j 0.054+0.175j│─────────────────────────────────│-0.943-0.311j  0.103+0.061j│───│ 0.059-0.587j  0.376+0.715j│───
           │0.018+0.182j 0.842+0.507j│                                 │-0.061+0.103j -0.676+0.727j│   │-0.758-0.278j -0.574+0.137j│
           └                         ┘                                 └                           ┘   └                           ┘
          └────────────────────────────────────────────────────────┘